In [1]:
# This is an application of the graphical directory parser on OCSSW's IDL directory.

import os
import networkx as nx
import matplotlib.pyplot as pl
from bokeh.models import ColumnDataSource
from math import sqrt
from bokeh.plotting import show, figure, output_file
from bokeh.io import output_notebook
from bokeh.models import HoverTool

In [2]:
% matplotlib inline

In [3]:
# NETWORK PARSING
def ParseDirTree2(tree=os.getcwd(),G=nx.Graph(),itr=0,maxItr=100,**kwds):
    colorDict = kwds.pop('colors',{'dir':'red','file':'magenta','init':'k'})
    if type(colorDict['dir']) == dict:
        dirColDict = colorDict['dir']
    else:
        dirColDict = None
    if type(colorDict['file']) == dict:
        filColDict = colorDict['file']
    else:
        filColDict = None
    subTree,leaves=[],[]
    if itr == 0:
        print(filColDict)
    itr += 1
    try:
        knot = tree.pop(0)
    except IndexError:
        print ("empty tree at itr=%d" % d)
    try:
        for x in os.listdir(knot):
            if x[0] != '.' and x[:2]!= '__':
                kx = os.path.join(knot,x)
                if os.path.isdir(kx):
                    subTree.append(kx)
                else:
                    leaves.append(kx)
    except FileNotFoundError as e:
        return G
    if subTree:
        G.add_edges_from(map(lambda b:(knot,b,{'weight':1/len(subTree)}), subTree))
        for node in subTree:
            if node[-1] == '/':
                node = node[:-1]
            lbl = os.path.abspath(node).split('/')[-1]
            G.node[node]['lbl'] = lbl
            if dirColDict:
                if lbl in dirColDict:
                    G.node[node]['color'] = dirColDict[lbl]
                else:
                    G.node[node]['color'] = dirColDict['other']
            else:
                G.node[node]['color'] = colorDict['dir']#'red'            
            # add capacity to detect whether colorDict['dir'] is a dict,
            # and if so, assume keys of dict are file extensions used to assign
            # color according to file type.
        tree.extend(subTree)
    if leaves:
        G.add_edges_from(map(lambda b:(knot,b,{'weight': 1/len(leaves)}),leaves))
        for node in leaves:
            lbl = os.path.abspath(node).split('/')[-1]
            ext = lbl.split('.')[-1]
            G.node[node]['lbl'] = lbl
            if filColDict:
                if ext in filColDict:
                    G.node[node]['color'] = filColDict[ext]
                else:
                    G.node[node]['color'] = filColDict['other']
            else:
                G.node[node]['color'] = colorDict['file']#'magenta'
            # add capacity to detect whether colorDict['file'] is a dict,
            # and if so assume keys of dict are file extensions used to assign
            # color according to file type.
    if itr == 1:
        if dirColDict:
            G.node[knot]['color'] = dirColDict['init']
        else:
            G.node[knot]['color'] = colorDict['init']
    G.node[knot]['lbl'] = os.path.abspath(knot).split('/')[-1]                
    if tree and itr<=maxItr:
        return ParseDirTree2(tree,G,itr,colors=colorDict)
    else:
        return G

In [4]:
# BOKEH NETWORK PLOTTING
def get_edges_specs(_network, _layout):
    d = dict(xs=[], ys=[], alphas=[])
    #weights = [d['weight'] for u, v, d in _network.edges(data=True)]
    max_weight = 1 #max(weights)
    calc_alpha = lambda h: 0.1 + 0.6 * (h / max_weight)
    # example: { ..., ('user47', 'da_bjoerni', {'weight': 3}), ... }
    for u, v, data in _network.edges(data=True):
        d['xs'].append([_layout[u][0], _layout[v][0]])
        d['ys'].append([_layout[u][1], _layout[v][1]])
        d['alphas'].append(calc_alpha(data['weight']))
    return d

In [5]:
def BokehDrawNetwork(G,nxLayout='spring',**kwargs):
    pK = .1/sqrt(G.number_of_nodes())
    nxLayout = kwargs.pop('nxLayout','spring')
    outFileName = kwargs.pop('outfile',None)
    figSize = kwargs.pop('figSize',(800,800))
    nodeSize = kwargs.pop('nodeSize',10)
    title = kwargs.pop('title',None)
    showMe = kwargs.pop('showMe',False)
    if nxLayout == 'spring':
        layout = nx.spring_layout(G, k=pK, iterations=100)
    nodes, nodes_coordinates = zip(*sorted(layout.items()))
    nodes_xs, nodes_ys = list(zip(*nodes_coordinates))
    colors = tuple([G.node[node]['color'] for node in nodes])
    lbl = tuple([G.node[node]['lbl'] for node in nodes])
    nodes_source = ColumnDataSource(dict(x=nodes_xs, y=nodes_ys,
                                        name=nodes,color=colors,label=lbl))
    hover = HoverTool(tooltips=[('label', '@label')])
    plotG = figure(title=title,plot_width=figSize[0], plot_height=figSize[1],
                   tools=['pan','tap',hover,'box_zoom','wheel_zoom','reset'],
                   title_text_font_size='16pt')
    r_circles = plotG.circle('x', 'y', source=nodes_source, size=nodeSize, color=colors,
                            level='overlay')
    line_source = ColumnDataSource(get_edges_specs(G,layout))
    r_lines = plotG.multi_line('xs', 'ys', line_width=1.5, alpha='alphas',
                             color='navy',source=line_source)
    
    if showMe:
        if outFileName:
            if '.html' not in outFileName:
                outFileName += '.html'
            output_file(outFileName,title=title)
        show(plotG)
    return plotG

In [6]:
root2 = ['/accounts/ekarakoy/ocssw/idl']

In [7]:
G_ocssw_idl = ParseDirTree2(root2,colors={'dir':{'init':'black','other':'brown'},
                               'file':{'pro':'red','idl':'orange','dat':'yellow',
                                      'other':'gray'}}) 

{'pro': 'red', 'idl': 'orange', 'dat': 'yellow', 'other': 'gray'}


In [9]:
output_notebook();

Loading BokehJS ...

In [10]:
plot_G_ocssw_idl=BokehDrawNetwork(G_ocssw_idl,title='ocssw/idl/',figureSize = (1200,1200),
                                 outFileName='ocssw_idl.html',showMe=True)
show(plot_G_ocssw_idl)

/accounts/ekarakoy/anaconda3/lib/python3.5/site-packages/bokeh/models/plots.py:712: UserWarning: 
    Plot property 'title_text_font_size' was deprecated in 0.12.0 and will be removed. Use 'Plot.title.text_font_size' instead.
    
  warnings.warn(DEP_MSG_0_12_0 % ('title_text_font_size', 'Plot.title.text_font_size'))


In [20]:
# EXPORT FOR GEPHI
nx.write_graphml(G_ocssw_idl,'idl.graphml')